In [1]:
#Loading data from csv file in S3
data = sqlContext.read.csv('s3a://decisiontreespark/fraud_detection_data.csv', header=True,inferSchema=True)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1638423848777_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

In [4]:
data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']

In [5]:
#select required columns
selectedColumns = ['step', 'type', 'amount',
                   'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
                   'newbalanceDest', 'isFraud', 'isFlaggedFraud']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#creating dataframe using selected columns
df = data.select(selectedColumns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+
|step|    type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0|        181.0|           0.0|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|      41554.0|      29885.86|           0.0|           0.0|      0|             0|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+
only showing top 5 rows

In [8]:
df.select('type').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|    type|
+--------+
| PAYMENT|
| PAYMENT|
|TRANSFER|
|CASH_OUT|
| PAYMENT|
+--------+
only showing top 5 rows

In [9]:
#group the transactions by 'type'
df.groupby('type').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+
|    type| count|
+--------+------+
|TRANSFER| 82424|
| CASH_IN|218673|
|CASH_OUT|362676|
| PAYMENT|329752|
|   DEBIT|  6474|
+--------+------+

In [10]:
#group the 'isFraud' to check fraud count
df.groupby('isFraud').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+
|isFraud| count|
+-------+------+
|      1|   535|
|      0|999464|
+-------+------+

In [11]:
print((df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(999999, 9)

### String Indexer

In [12]:
from pyspark.ml.feature import StringIndexer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
si = StringIndexer(inputCol='type', outputCol='type_index')
df_indexed = si.fit(df).transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_indexed.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+
|step|    type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+
|   1| PAYMENT| 9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|
|   1| PAYMENT| 1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|
|   1|TRANSFER|   181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|
|   1|CASH_OUT|   181.0|        181.0|           0.0|       21182.0|           0.0|      1|             0|       0.0|
|   1| PAYMENT|11668.14|      41554.0|      29885.86|           0.0|           0.0|      0|             0|       1.0|
+----+--------+--------+-------------+--------------+---

### One Hot Encoder

In [17]:
from pyspark.ml.feature import OneHotEncoderEstimator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
encoder = OneHotEncoderEstimator(inputCols = ['type_index'], outputCols=['type_encoded'])
df_enc = encoder.fit(df_indexed).transform(df_indexed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_enc.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+
|step|    type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+
|   1| PAYMENT| 9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|
|   1| PAYMENT| 1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|
|   1|TRANSFER|   181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|(4,[3],[1.0])|
|   1|CASH_OUT|   181.0|        181.0|           0.0|       21182.0|           0.0|      1|             0|       0.0|(4,[0],[1.0])|
|   1| PAYMENT|11668.14|      41554.0|      29885.86|           0.0|        

### Vecotor Assembler

In [21]:
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_enc.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud', 'type_index', 'type_encoded']

In [24]:
#select the required Columns
selected_columns = ['step', 'amount', 'oldbalanceOrg', 
                 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest',  
                 'type_encoded']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
va = VectorAssembler(inputCols = selected_columns, outputCol = 'features')
df_vectors = va.transform(df_enc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df_vectors.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+
|step|    type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|            features|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+
|   1| PAYMENT| 9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|(10,[0,1,2,3,7],[...|
|   1| PAYMENT| 1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|(10,[0,1,2,3,7],[...|
|   1|TRANSFER|   181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|(4,[3],[1.0])|(10,[0,1,2,9],[1....|
|   1|CASH_OUT|   181.0|        181.0|           0.0|       21182.0|           0.0

### Create Train Test Split

In [27]:
(trainingData, testData) = df_vectors.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
trainingData.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

700090

In [30]:
testData.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

299598

### Decision Tree Clasifier

In [33]:
from pyspark.ml.classification import DecisionTreeClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol='isFraud')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
model = dt.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
predictions = model.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
predictions.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+----------------+--------------------+----------+
|step|   type| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|            features|   rawPrediction|         probability|prediction|
+----+-------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+----------------+--------------------+----------+
|   1|CASH_IN| 484.57|   5422437.76|    5422922.33|    5638778.53|    5579568.65|      0|             0|       2.0|(4,[2],[1.0])|[1.0,484.57,54224...|[642245.0,152.0]|[0.99976338619265...|       0.0|
|   1|CASH_IN|1271.77|    6973823.5|    6975095.27|     697456.73|    2719172.89|      0|             0|       2.0|(4,[2],[1.0])|[1.0,1271.77,6973...|[642245.0,152.0]|[0.99976338619265...|       0.0|


In [41]:
predictions.select('prediction', 'isFraud', 'features').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+--------------------+
|prediction|isFraud|            features|
+----------+-------+--------------------+
|       0.0|      0|[1.0,484.57,54224...|
|       0.0|      0|[1.0,1271.77,6973...|
|       0.0|      0|[1.0,2673.64,7688...|
|       0.0|      0|[1.0,6284.18,7858...|
|       0.0|      0|[1.0,7322.98,6681...|
+----------+-------+--------------------+
only showing top 5 rows

### Accuracy

In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
evaluator = MulticlassClassificationEvaluator(labelCol='isFraud', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
accuracy

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9996899059381346

In [45]:
predictions.groupby('isFraud').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+
|isFraud| count|
+-------+------+
|      1|   142|
|      0|299456|
+-------+------+

In [46]:
predictions.groupby('prediction').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+
|prediction| count|
+----------+------+
|       0.0|299494|
|       1.0|   104|
+----------+------+

### Confusion Matrix

In [47]:
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import MulticlassMetrics
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
preds_and_labels = predictions.select(['prediction','isFraud']).withColumn('label', F.col('isFraud').cast(FloatType())).orderBy('prediction')

preds_and_labels = preds_and_labels.select(['prediction','label'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
preds_and_labels.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows

In [51]:
#convert into RDD
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
#print in form of an array
print(metrics.confusionMatrix().toArray())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[2.99433e+05 2.30000e+01]
 [6.10000e+01 8.10000e+01]]

### Precision

In [53]:
precision = metrics.precision()
precision

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9997196242965574

### Recall

In [54]:
recall = metrics.recall()
recall

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9997196242965574

### F1 Score

In [55]:
f1Score = metrics.fMeasure()
f1Score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9997196242965574